In [228]:
import pandas as pd
import matplotlib.pyplot as plt


In [229]:
df = pd.read_csv(
    "../data/raw/50000 Sales Records.csv",
    parse_dates=["Order Date", "Ship Date"]
)


In [230]:
print(df.head())
print(df.info())
print(df.describe())


               Region   Country  Item Type Sales Channel Order Priority  \
0  Sub-Saharan Africa   Namibia  Household       Offline              M   
1              Europe   Iceland  Baby Food        Online              H   
2              Europe    Russia       Meat        Online              L   
3              Europe  Moldova        Meat        Online              L   
4              Europe     Malta     Cereal        Online              M   

  Order Date   Order ID  Ship Date  Units Sold  Unit Price  Unit Cost  \
0 2015-08-31  897751939 2015-10-12        3604      668.27     502.54   
1 2010-11-20  599480426 2011-01-09        8435      255.28     159.42   
2 2017-06-22  538911855 2017-06-25        4848      421.89     364.69   
3 2012-02-28  459845054 2012-03-20        7225      421.89     364.69   
4 2010-08-12  626391351 2010-09-13        1975      205.70     117.11   

   Total Revenue  Total Cost  Total Profit  
0     2408445.08  1811154.16     597290.92  
1     2153286.80  13

In [231]:
print(df.isna().sum())


Region            0
Country           0
Item Type         0
Sales Channel     0
Order Priority    0
Order Date        0
Order ID          0
Ship Date         0
Units Sold        0
Unit Price        0
Unit Cost         0
Total Revenue     0
Total Cost        0
Total Profit      0
dtype: int64


In [232]:
df = df.rename(columns={
    "Order Date": "date",
    "Total Revenue": "sales",
    "Country": "store_id",
    "Sales Channel": "sales_channel"
})

In [233]:
df = df[["date", "store_id", "sales", "sales_channel", "Item Type"]]



In [234]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month


In [235]:
monthly_df = (
    df.groupby(["store_id", "year", "month"])["sales"]
      .sum()
      .reset_index()
)

In [236]:
monthly_df["date"] = pd.to_datetime(
    monthly_df["year"].astype(str) + "-" +
    monthly_df["month"].astype(str) + "-01"
) + pd.offsets.MonthEnd(0)

monthly_df = monthly_df[["date", "store_id", "sales"]]

print(monthly_df.head())
print(monthly_df.shape)


        date     store_id       sales
0 2010-02-28  Afghanistan  5496863.61
1 2010-03-31  Afghanistan  2972557.16
2 2010-04-30  Afghanistan  3597418.95
3 2010-05-31  Afghanistan  4777412.05
4 2010-06-30  Afghanistan   792721.45
(16035, 3)


In [237]:
monthly_df.to_csv("../data/processed/monthly_sales.csv", index=False)


In [238]:
monthly_df.head()


,date,store_id,sales
0,2010-02-28,Afghanistan,5496863.61
1,2010-03-31,Afghanistan,2972557.16
2,2010-04-30,Afghanistan,3597418.95
3,2010-05-31,Afghanistan,4777412.05
4,2010-06-30,Afghanistan,792721.45


In [239]:
monthly_df = monthly_df.sort_values(by=["store_id", "date"])
print(monthly_df.head())
print(monthly_df.tail())


        date     store_id       sales
0 2010-02-28  Afghanistan  5496863.61
1 2010-03-31  Afghanistan  2972557.16
2 2010-04-30  Afghanistan  3597418.95
3 2010-05-31  Afghanistan  4777412.05
4 2010-06-30  Afghanistan   792721.45
            date  store_id        sales
16030 2017-03-31  Zimbabwe   7979961.37
16031 2017-04-30  Zimbabwe   2435992.86
16032 2017-05-31  Zimbabwe  10372448.54
16033 2017-06-30  Zimbabwe   4862800.02
16034 2017-07-31  Zimbabwe    827899.08


In [240]:
df = monthly_df.copy()

df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["quarter"] = df["date"].dt.quarter
df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)

df["is_month_start"] = df["date"].dt.is_month_start.astype(int)
df["is_month_end"] = df["date"].dt.is_month_end.astype(int)

print(df.head())


        date     store_id       sales  year  month  quarter  weekofyear  \
0 2010-02-28  Afghanistan  5496863.61  2010      2        1           8   
1 2010-03-31  Afghanistan  2972557.16  2010      3        1          13   
2 2010-04-30  Afghanistan  3597418.95  2010      4        2          17   
3 2010-05-31  Afghanistan  4777412.05  2010      5        2          22   
4 2010-06-30  Afghanistan   792721.45  2010      6        2          26   

   is_month_start  is_month_end  
0               0             1  
1               0             1  
2               0             1  
3               0             1  
4               0             1  


In [241]:
#We Created Lag past memory of sales
for lag in [1, 2, 3, 6, 12]:
    df[f"lag_{lag}"] = df.groupby("store_id")["sales"].shift(lag)


In [242]:
df.head(15)

,date,store_id,sales,year,month,quarter,weekofyear,is_month_start,is_month_end,lag_1,lag_2,lag_3,lag_6,lag_12
0,2010-02-28,Afghanistan,5496863.61,2010,2,1,8,0,1,NaN,NaN,NaN,NaN,NaN
1,2010-03-31,Afghanistan,2972557.16,2010,3,1,13,0,1,5496863.61,NaN,NaN,NaN,NaN
2,2010-04-30,Afghanistan,3597418.95,2010,4,2,17,0,1,2972557.16,5496863.61,NaN,NaN,NaN
3,2010-05-31,Afghanistan,4777412.05,2010,5,2,22,0,1,3597418.95,2972557.16,5496863.61,NaN,NaN
4,2010-06-30,Afghanistan,792721.45,2010,6,2,26,0,1,4777412.05,3597418.95,2972557.16,NaN,NaN
5,2010-07-31,Afghanistan,8745750.30,2010,7,3,30,0,1,792721.45,4777412.05,3597418.95,NaN,NaN
6,2010-08-31,Afghanistan,10013486.68,2010,8,3,35,0,1,8745750.30,792721.45,4777412.05,5496863.61,NaN
7,2010-09-30,Afghanistan,2294711.92,2010,9,3,39,0,1,10013486.68,8745750.30,792721.45,2972557.16,NaN
8,2010-10-31,Afghanistan,9951809.87,2010,10,4,43,0,1,2294711.92,10013486.68,8745750.30,3597418.95,NaN
9,2010-11-30,Afghanistan,2122398.15,2010,11,4,48,0,1,9951809.87,2294711.92,10013486.68,4777412.05,NaN


In [243]:
# why lag important: Forecasting is mostly learned from past values of sales itself.Rolling mean = smoothed past trend

In [244]:
df["rolling_3_mean"] = (
    df.groupby("store_id")["sales"]
      .shift(1)
      .rolling(window=3)
      .mean()
)

df["rolling_6_mean"] = (
    df.groupby("store_id")["sales"]
      .shift(1)
      .rolling(window=6)
      .mean()
)


In [245]:
# Why rolling means help:
# Removes noise
# Helps model understand trend direction
# Used heavily in real retail forecasting

In [246]:
df = df.dropna().reset_index(drop=True)

print(df.isna().sum())


date              0
store_id          0
sales             0
year              0
month             0
quarter           0
weekofyear        0
is_month_start    0
is_month_end      0
lag_1             0
lag_2             0
lag_3             0
lag_6             0
lag_12            0
rolling_3_mean    0
rolling_6_mean    0
dtype: int64


In [247]:
df.to_csv("../data/processed/features.csv", index=False)


In [248]:
df = pd.read_csv("../data/processed/features.csv", parse_dates=["date"])

print(df.columns)
print(df.head())

Index(['date', 'store_id', 'sales', 'year', 'month', 'quarter', 'weekofyear',
       'is_month_start', 'is_month_end', 'lag_1', 'lag_2', 'lag_3', 'lag_6',
       'lag_12', 'rolling_3_mean', 'rolling_6_mean'],
      dtype='object')
        date     store_id        sales  year  month  quarter  weekofyear  \
0 2011-02-28  Afghanistan   4097431.35  2011      2        1           9   
1 2011-03-31  Afghanistan       746.40  2011      3        1          13   
2 2011-04-30  Afghanistan   2557038.88  2011      4        2          17   
3 2011-05-31  Afghanistan  10653528.50  2011      5        2          22   
4 2011-06-30  Afghanistan   2691529.56  2011      6        2          26   

   is_month_start  is_month_end        lag_1       lag_2       lag_3  \
0               0             1   2392876.14  2758174.44  2122398.15   
1               0             1   4097431.35  2392876.14  2758174.44   
2               0             1       746.40  4097431.35  2392876.14   
3               0       

In [249]:
import pandas as pd

df = pd.read_csv("../data/processed/features.csv", parse_dates=["date"])

print(df.shape)
df.head()
#Time-Based Train–Validation Split - We must never do random split in forecasting

(13815, 16)


,date,store_id,sales,year,month,quarter,weekofyear,is_month_start,is_month_end,lag_1,lag_2,lag_3,lag_6,lag_12,rolling_3_mean,rolling_6_mean
0,2011-02-28,Afghanistan,4097431.35,2011,2,1,9,0,1,2392876.14,2758174.44,2122398.15,10013486.68,5496863.61,2.424483e+06,4.922243e+06
1,2011-03-31,Afghanistan,746.40,2011,3,1,13,0,1,4097431.35,2392876.14,2758174.44,2294711.92,2972557.16,3.082827e+06,3.936234e+06
2,2011-04-30,Afghanistan,2557038.88,2011,4,2,17,0,1,746.40,4097431.35,2392876.14,9951809.87,3597418.95,2.163685e+06,3.553906e+06
3,2011-05-31,Afghanistan,10653528.50,2011,5,2,22,0,1,2557038.88,746.40,4097431.35,2122398.15,4777412.05,2.218406e+06,2.321444e+06
4,2011-06-30,Afghanistan,2691529.56,2011,6,2,26,0,1,10653528.50,2557038.88,746.40,2758174.44,792721.45,4.403771e+06,3.743299e+06


In [250]:
obj_cols = df.select_dtypes(include="object").columns.tolist()
print("Object columns:", obj_cols)

for col in obj_cols:
    df[col] = df[col].astype("category").cat.codes


Object columns: ['store_id']


In [251]:
df = df.sort_values(by="date")
last_n_months = 6
max_date = df["date"].max()
cutoff_date = max_date - pd.DateOffset(months=last_n_months)

train_df = df[df["date"] <= cutoff_date]
valid_df = df[df["date"] > cutoff_date]

print("Train shape:", train_df.shape)
print("Valid shape:", valid_df.shape)


Train shape: (12762, 16)
Valid shape: (1053, 16)


In [252]:
store_summary = df.groupby("store_id").agg(
    avg_sales=("sales", "mean"),
    volatility=("sales", "std"),
    max_sales=("sales", "max")
).reset_index()

print(store_summary.head())


   store_id     avg_sales    volatility    max_sales
0         0  4.148436e+06  3.052758e+06  11211403.32
1         1  4.348804e+06  2.972690e+06  11057912.75
2         2  3.539948e+06  2.894383e+06  11242582.48
3         3  4.284236e+06  3.318764e+06  13383064.54
4         4  4.223241e+06  3.070592e+06  13518736.62


In [253]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler = StandardScaler()
X_cluster = scaler.fit_transform(
    store_summary[["avg_sales", "volatility", "max_sales"]]
)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
store_summary["cluster"] = kmeans.fit_predict(X_cluster)

print(store_summary.head())


   store_id     avg_sales    volatility    max_sales  cluster
0         0  4.148436e+06  3.052758e+06  11211403.32        2
1         1  4.348804e+06  2.972690e+06  11057912.75        2
2         2  3.539948e+06  2.894383e+06  11242582.48        2
3         3  4.284236e+06  3.318764e+06  13383064.54        0
4         4  4.223241e+06  3.070592e+06  13518736.62        0


In [254]:
df = df.merge(
    store_summary[["store_id", "cluster"]],
    on="store_id",
    how="left"
)

print(df[["store_id", "date", "sales", "cluster"]].head())


   store_id       date       sales  cluster
0       113 2011-01-31  2388927.54        0
1       142 2011-01-31  4348646.05        0
2       165 2011-01-31  1192031.50        0
3       141 2011-01-31  7322812.73        2
4       140 2011-01-31  6213845.82        0


In [255]:
clusters = sorted(df["cluster"].unique())
print("Clusters:", clusters)


Clusters: [np.int32(0), np.int32(1), np.int32(2)]


In [256]:
df.to_csv("../data/processed/features_with_clusters.csv", index=False)
print("Saved features_with_clusters.csv")


Saved features_with_clusters.csv


In [257]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import joblib


In [258]:
df = pd.read_csv("../data/processed/features_with_clusters.csv", parse_dates=["date"])

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())


Shape: (13815, 17)
Columns: ['date', 'store_id', 'sales', 'year', 'month', 'quarter', 'weekofyear', 'is_month_start', 'is_month_end', 'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12', 'rolling_3_mean', 'rolling_6_mean', 'cluster']


In [260]:
df = df.sort_values(by="date")


In [261]:
last_n_months = 6
max_date = df["date"].max()
cutoff_date = max_date - pd.DateOffset(months=last_n_months)

train_df = df[df["date"] <= cutoff_date]
valid_df = df[df["date"] > cutoff_date]

print("Train shape:", train_df.shape)
print("Valid shape:", valid_df.shape)


Train shape: (12762, 17)
Valid shape: (1053, 17)


In [262]:
TARGET = "sales"
EXCLUDE_COLS = ["sales", "date", "cluster"]

feature_cols = [c for c in df.columns if c not in EXCLUDE_COLS]
print("Features used:", feature_cols)

clusters = sorted(df["cluster"].unique())
print("Clusters:", clusters)



Features used: ['store_id', 'year', 'month', 'quarter', 'weekofyear', 'is_month_start', 'is_month_end', 'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12', 'rolling_3_mean', 'rolling_6_mean']
Clusters: [np.int64(0), np.int64(1), np.int64(2)]


In [266]:
cluster_results = []
for cl in clusters:
    print(f"\n🔹 Training model for cluster {cl}")
    train_c = train_df[train_df["cluster"] == cl]
    valid_c = valid_df[valid_df["cluster"] == cl]
    print("  Train rows:", train_c.shape[0], " Valid rows:", valid_c.shape[0])
    if valid_c.shape[0] < 10:
        print("  ⚠️ Skipping cluster (too few validation rows).")
        continue
    X_train_c = train_c[feature_cols]
    y_train_c = train_c[TARGET]
    X_valid_c = valid_c[feature_cols]
    y_valid_c = valid_c[TARGET]
    model_c = RandomForestRegressor(
        n_estimators=200,
        max_depth=8,
        random_state=42,
        n_jobs=-1
    )
    model_c.fit(X_train_c, y_train_c) 
    y_pred_c = model_c.predict(X_valid_c)
    mae_c = mean_absolute_error(y_valid_c, y_pred_c)
    rmse_c = sqrt(mean_squared_error(y_valid_c, y_pred_c))
    print(f"  Cluster {cl} MAE : {mae_c:,.2f}")
    print(f"  Cluster {cl} RMSE: {rmse_c:,.2f}")
    model_path = f"../models/xgb_cluster_{cl}.pkl"
    joblib.dump(model_c, model_path)
    print(f"  Saved model to {model_path}")
    
    cluster_results.append({
        "cluster": cl,
        "mae": mae_c,
        "rmse": rmse_c,
        "valid_rows": valid_c.shape[0]
    })

print("\n✅ Finished training all available clusters.")

    



🔹 Training model for cluster 0
  Train rows: 6272  Valid rows: 517
  Cluster 0 MAE : 2,519,950.69
  Cluster 0 RMSE: 3,168,695.12
  Saved model to ../models/xgb_cluster_0.pkl

🔹 Training model for cluster 1
  Train rows: 2911  Valid rows: 241
  Cluster 1 MAE : 2,870,688.48
  Cluster 1 RMSE: 3,541,428.68
  Saved model to ../models/xgb_cluster_1.pkl

🔹 Training model for cluster 2
  Train rows: 3579  Valid rows: 295
  Cluster 2 MAE : 2,523,484.33
  Cluster 2 RMSE: 3,030,615.57
  Saved model to ../models/xgb_cluster_2.pkl

✅ Finished training all available clusters.


In [267]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

baseline_pred = valid_df["lag_1"]

baseline_mae = mean_absolute_error(valid_df["sales"], baseline_pred)
baseline_rmse = sqrt(mean_squared_error(valid_df["sales"], baseline_pred))

print("✅ Baseline MAE :", round(baseline_mae, 2))
print("✅ Baseline RMSE:", round(baseline_rmse, 2))


✅ Baseline MAE : 3583565.22
✅ Baseline RMSE: 4593886.35


In [268]:
total_weight = 0
weighted_mae = 0

for res in cluster_results:
    weighted_mae += res["mae"] * res["valid_rows"]
    total_weight += res["valid_rows"]

rf_global_mae = weighted_mae / total_weight

print("✅ RandomForest Global MAE:", round(rf_global_mae, 2))


✅ RandomForest Global MAE: 2601213.97


In [269]:
improvement_percent = ((baseline_mae - rf_global_mae) / baseline_mae) * 100

print("Accuracy Improvement %:", round(improvement_percent, 2), "%")


Accuracy Improvement %: 27.41 %
